## Face Detection with Count and Distance Measurement
### Importing modules

### Selecting which camera to use

In [1]:
import numpy as np
#numerical calculations
from scipy.spatial import distance as dist
#scientific calculations

In [2]:
import cv2 #computer vision library


In [3]:
cv2.__version__


'3.4.2'

In [4]:
cap=cv2.VideoCapture(0)  #0 is webcam

cap = cv2.VideoCapture(0)   

In [14]:

face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


### Capturing the photo and showing image

In [8]:
status , photo = cap.read() # two values 1st value will be status:whether the camera is on are not,
#2nd statement photo whgich is captured
cv2.imshow('Socail Distancing' , photo)
cv2.waitKey()
cv2.destroyAllWindows()



## Capture video and Show
### Count and measurment logic

In [20]:
sudo apt install libgtk2.0-dev pkg-config

SyntaxError: invalid syntax (<ipython-input-20-04dcd4529ffb>, line 1)

In [ ]:
while True: 
    status , photo = cap.read() 
    face_cor = face_model.detectMultiScale(photo)
    l = len(face_cor)
    photo = cv2.putText(photo, str(len(face_cor))+" Face", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (255, 0, 0) , 2, cv2.LINE_AA) 
    stack_x = [] 
    stack_y = []
    stack_x_print = [] 
    stack_y_print = []
    global D
    
    if len(face_cor) == 0:
        pass
    
    else:
        for i in range(0,len(face_cor)):
            x1 = face_cor[i][0]
            y1 = face_cor[i][1]
            x2 = face_cor[i][0] + face_cor[i][2]
            y2 = face_cor[i][1] + face_cor[i][3]
            
             
            
            mid_x = int((x1+x2)/2)
            mid_y = int((y1+y2)/2)
            stack_x.append(mid_x)
            stack_y.append(mid_y)
            stack_x_print.append(mid_x)
            stack_y_print.append(mid_y)
            
            photo = cv2.circle(photo, (mid_x, mid_y), 3 , [255,0,0] , -1) 
            photo = cv2.rectangle(photo , (x1, y1) , (x2,y2) , [0,255,0] , 2)
        
        if len(face_cor) == 2:
            D = int(dist.euclidean((stack_x.pop(), stack_y.pop()), (stack_x.pop(), stack_y.pop())))
            photo = cv2.line(photo, (stack_x_print.pop(), stack_y_print.pop()), (stack_x_print.pop(), stack_y_print.pop()), [0,0,255], 2)
        else:
            D = 0
        
        if D<250 and D!=0:
            photo = cv2.putText(photo, "!!MOVE AWAY!!", (120, 100), cv2.FONT_HERSHEY_SIMPLEX,2, [0,0,255] , 4)
            
        photo = cv2.putText(photo, str(D/10) + " cm", (300, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (255, 0, 0) , 2, cv2.LINE_AA)
        
        cv2.imshow('hi' , photo)
        if cv2.waitKey(100) == 13:
            break
            
cv2.destroyAllWindows()

### Releasing the Camera

In [9]:
cap.release()


## Second part of the Project for CCTV analysis

In [5]:
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import os

In [ ]:
!pip install imutils

In [6]:
# base path to YOLO directory
MODEL_PATH = "/social_distance_analyzer-master/social_distance_analyzer-master/yolo-coco"

# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# boolean indicating if NVIDIA CUDA GPU should be used
USE_GPU = False

# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50

In [7]:
# load the COCO class labels our YOLO model was trained on
labelsPath = 'social_distance_analyzer-master/social_distance_analyzer-master/yolo-coco/coco.names'
LABELS = open(labelsPath).read().strip().split("\n")

print(LABELS)

print(len(LABELS))

# derive the paths to the YOLO weights and model configuration
weightsPath = 'social_distance_analyzer-master/social_distance_analyzer-master/yolo-coco/yolov3.weights'
configPath = 'social_distance_analyzer-master/social_distance_analyzer-master/yolo-coco/yolov3.cfg'

# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80
[INFO] loading YOLO from disk...


In [8]:
if USE_GPU:
    # set CUDA as the preferable backend and target
    print("Setting preferable backend and target to CUDA...")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [9]:
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

## Section 3: Input access (Video file and Live footage)
The following section concentrates on building a block that accepts already existing video file or a live footage as an input for the algorithm.

[Note : Use the below block or the block after that dont use both]

In [11]:
#upload the video file you want to check social distancing for- below.
vs = cv2.VideoCapture('social_distance_analyzer-master/social_distance_analyzer-master/5_6127594218744447539.mp4')
#src.upload(vs)
fps = vs.get(cv2.CAP_PROP_FPS)
print("FPS of the current video: ",fps)

num_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
print("Number of frames in the video: ",num_frames)

writer = None

FPS of the current video:  0.0
Number of frames in the video:  0.0


### Section 4: Algorithm Detecting people¶


In [ ]:
def detect_people(frame, net, ln, personIdx=0):

    # grab the dimensions of the frame and  initialize the list of
    # results
    (H, W) = frame.shape[:2]
    results = []

    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # initialize our lists of detected bounding boxes, centroids, and
    # confidences, respectively
    boxes = []
    centroids = []
    confidences = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter detections by (1) ensuring that the object
            # detected was a person and (2) that the minimum
            # confidence is met
            if classID == personIdx and confidence > MIN_CONF:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # centroids, and confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # update our results list to consist of the person
            # prediction probability, bounding box coordinates,
            # and the centroid
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    # return the list of results
    return results